# Problem 2a

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
import scipy.stats as sts

macro = np.loadtxt("/Users/benjaminlim/Documents/BootCamp2017/Labs/Econ/Week 4/MacroSeries.txt", delimiter = ",")
w = macro[:, 2]
k = macro[:, 1]
r = macro[:, 3]

In [2]:
#normal pdf for random variable z
def truncnorm_pdf(w, k, alpha, rho, mu, sigma):
    z = np.log(w/((k**alpha)*(1-alpha)))
    zprime = np.append([mu],z)
    zprime = zprime[:-1]

    mean = rho*zprime + (1-rho)*mu
    pdf_vals = sts.norm.pdf(z, loc = mean, scale = sigma)
    
    return pdf_vals

truncnorm_pdf(w, k, 0.35, 0.9, 3, 1)

array([  3.61868860e-15,   2.62312865e-01,   3.42320190e-01,
         3.10295515e-01,   2.74262283e-01,   2.90645432e-01,
         2.83607774e-01,   2.84311728e-01,   2.99749884e-01,
         2.65523158e-01,   2.89317113e-01,   2.99168119e-01,
         2.58415731e-01,   2.49368026e-01,   3.29621460e-01,
         2.96741214e-01,   2.75846469e-01,   3.07237024e-01,
         3.29612323e-01,   2.52257107e-01,   3.02784681e-01,
         3.09485783e-01,   3.10963075e-01,   2.71528149e-01,
         2.82985381e-01,   3.09992451e-01,   3.06671195e-01,
         2.57262941e-01,   2.82894993e-01,   3.00119631e-01,
         2.78175622e-01,   2.66809713e-01,   2.73186338e-01,
         2.98267163e-01,   2.19502194e-01,   2.59263205e-01,
         2.97887126e-01,   2.68461433e-01,   2.91129546e-01,
         3.39323563e-01,   3.14780275e-01,   3.06447097e-01,
         2.80710920e-01,   3.07412843e-01,   2.89337955e-01,
         2.75061823e-01,   2.88941342e-01,   3.10624312e-01,
         2.84043962e-01,

In [6]:
def log_lik_trnorm(w, k, alpha, rho, mu, sigma):

    pdf_vals = truncnorm_pdf(w, k, alpha, rho, mu, sigma)
    pdf_vals[np.isclose(pdf_vals, np.zeros_like(pdf_vals), atol = 1e-25)] = 1e-10
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit(params, w, k):

    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_trnorm(w, k, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

import scipy.optimize as opt

alpha_init = 0.457
rho_init = 0.72
mu_init = 9.52
sigma_init = 1
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
mle_args = (w, k)
results = opt.minimize(crit, params_init, args=(mle_args), method='L-BFGS-B', bounds=((1e-10, 1-1e-10),(1e-10-1, 1-1e-10), (1e-10, None), (1e-10, None)))
alpha_hat, rho_hat, mu_hat, sigma_hat = results.x
print("alpha_hat: ", alpha_hat, "rho_hat: ", rho_hat, "mu_hat", mu_hat, "sigma_hat", sigma_hat)
mle_hat = log_lik_trnorm(w, k, alpha_hat, rho_hat, mu_hat, sigma_hat)
print("Log-likelihood: ", mle_hat)
vcv1 = results.hess_inv.todense()
print(vcv1)
print(results)

alpha_hat:  0.457506737794 rho_hat:  0.720497905433 mu_hat 9.5228554852 sigma_hat 0.091996266099
Log-likelihood:  96.7069080597
[[  2.31592945e+02  -1.51938600e+02  -3.29755420e+03  -4.10580302e+00]
 [ -1.51938600e+02   1.00074072e+02   2.16261055e+03   2.65153315e+00]
 [ -3.29755420e+03   2.16261055e+03   4.69540730e+04   5.85476279e+01]
 [ -4.10580302e+00   2.65153315e+00   5.85476279e+01   7.76066951e-02]]
      fun: -96.706908059741281
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0278817 ,  0.00030695,  0.00198526,  0.00082565])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 225
      nit: 23
   status: 0
  success: True
        x: array([ 0.45750674,  0.72049791,  9.52285549,  0.09199627])


# Problem 2b

In [7]:
#normal pdf for random variable z

def truncnorm_pdf2(r, k, alpha, rho, mu, sigma):
    denom = alpha * (k ** (alpha - 1))
    z = np.log(r / denom)    
    zprime = np.append([mu],z)
    zprime = zprime[:-1]

    mean = rho*zprime + (1-rho)*mu
    pdf_vals = (1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z - mean)**2 / (2 * sigma**2)))
    pdf_vals[np.isclose(pdf_vals, np.zeros_like(pdf_vals), atol = 1e-25)] = 1e-10    
    return pdf_vals

def log_lik_trnorm2(r, k, alpha, rho, mu, sigma):
    pdf_vals = truncnorm_pdf2(r, k, alpha, rho, mu, sigma)
    pdf_vals[np.isclose(pdf_vals, np.zeros_like(pdf_vals), atol = 1e-25)] = 1e-10   
    ln_pdf_vals =   np.log(pdf_vals, where=(pdf_vals>0))
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit2(params, *args):
    alpha, rho, mu, sigma = params
    r, k = args
    log_lik_val = log_lik_trnorm2(r, k, alpha, rho, mu, sigma)
    return -log_lik_val

alpha_init = 0.5
rho_init = 0.9
mu_init = 5
sigma_init = 1
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
mle_args_2 = (r, k)
results2 = opt.minimize(crit2, params_init, args=(mle_args_2), method='L-BFGS-B',
                       bounds=((1e-10, 1-1e-10),(1e-10-1, 1-1e-10),
                               (1e-10, None), (1e-10, None)))

alpha_hat_2, rho_hat_2, mu_hat_2, sigma_hat_2 = results2.x
print("alpha_hat: ", alpha_hat_2, "rho_hat: ", rho_hat_2, "mu_hat", mu_hat_2, "sigma_hat", sigma_hat_2)
print("Log-likelihood: ", log_lik_trnorm2(r, k, alpha_hat_2, rho_hat_2, mu_hat_2, sigma_hat_2))
vcv2 = results2.hess_inv.todense()
print(vcv2)


alpha_hat:  0.457500784381 rho_hat:  0.72050253195 mu_hat 9.37057284326 sigma_hat 0.0919961812852
Log-likelihood:  96.7069080537
[[  4.41070599e+01  -2.45620877e+01  -7.94380670e+02  -5.39378835e-01]
 [ -2.45620877e+01   1.43114444e+01   4.42560356e+02   2.99590646e-01]
 [ -7.94380670e+02   4.42560356e+02   1.43070799e+04   9.71414921e+00]
 [ -5.39378835e-01   2.99590646e-01   9.71414921e+00   6.64512723e-03]]


# Problem 2c

In [11]:
zstar = np.log(1/(alpha_hat*(7500000**(alpha_hat-1))))

prob = 1 - sts.norm.cdf(zstar, 10*rho_hat + (1 - rho_hat)*mu_hat, sigma_hat )
print("Probability that interest rate is greater than 1 = ", prob)

Probability that interest rate is greater than 1 =  0.999999967117
